# Exploratory Data Analysis

In [36]:
import pandas as pd
import numpy as np

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print(train.head())
print('--------------------------------------------------------------------------------')
print(test.head())

  portfolio_id      desk_id    office_id pf_category  start_date         sold  \
0   PF00001002  DSK00001001  OFF00001002           B    20040720  110000000.0   
1   PF00001003  DSK00001002  OFF00001001           A    20040709  176671000.0   
2   PF00001005  DSK00001004  OFF00001001           A    20040723   56474000.0   
3   PF00001006  DSK00001005  OFF00001001           A    20040609  164813000.0   
4   PF00001007  DSK00001005  OFF00001002           B    20040609  140800000.0   

  country_code  euribor_rate currency  libor_rate        bought  \
0            T       0.02074      USD    2.332216  1.098097e+08   
1            N       0.02074      GBP    5.269617  1.760084e+08   
2            T       0.02074      USD    2.332216  5.637953e+07   
3            T       0.02074      USD    2.332216  1.645088e+08   
4            T       0.02074      USD    2.332216  1.405402e+08   

   creation_date indicator_code  sell_date type hedge_value status   return  
0       20040720            NaN 

In [37]:
# Shapes and types of features
print('train Shape:', train.shape, 'test Shape:', test.shape)
print('---------------------------------')
print('train FEATURE TYPES\n\n', train.dtypes)
print('---------------------------------')
print('test FEATURE TYPES\n\n', test.dtypes)

train Shape: (9366, 18) test Shape: (4801, 17)
---------------------------------
train FEATURE TYPES

 portfolio_id       object
desk_id            object
office_id          object
pf_category        object
start_date          int64
sold              float64
country_code       object
euribor_rate      float64
currency           object
libor_rate        float64
bought            float64
creation_date       int64
indicator_code     object
sell_date           int64
type               object
hedge_value        object
status             object
return            float64
dtype: object
---------------------------------
test FEATURE TYPES

 portfolio_id       object
desk_id            object
office_id          object
pf_category        object
start_date          int64
sold              float64
country_code       object
euribor_rate      float64
currency           object
libor_rate        float64
bought            float64
creation_date       int64
indicator_code     object
sell_date           in

In [38]:
# Number of missing values
print("train MISSING VALUES\n\n", train.isnull().sum())
print('---------------------------------')
print("test MISSING VALUES \n\n", test.isnull().sum())

train MISSING VALUES

 portfolio_id         0
desk_id           3665
office_id            0
pf_category          0
start_date           0
sold                 2
country_code         0
euribor_rate         0
currency             0
libor_rate         474
bought               2
creation_date        0
indicator_code    5699
sell_date            0
type                 0
hedge_value       5701
status            3084
return               0
dtype: int64
---------------------------------
test MISSING VALUES 

 portfolio_id         0
desk_id           1948
office_id            0
pf_category          0
start_date           0
sold                 0
country_code         0
euribor_rate         0
currency             0
libor_rate         265
bought               0
creation_date        0
indicator_code    2851
sell_date            0
type                 0
hedge_value       2851
status            1457
dtype: int64


In [39]:
# Look at the description of the training data's object features
train.describe(include = [np.object])

,portfolio_id,desk_id,office_id,pf_category,country_code,currency,indicator_code,type,hedge_value,status
count,9366,5701,9366,9366,9366,9366,3667,9366,3665,6282
unique,9366,3992,2,5,5,5,1,8,2,1
top,PF00003545,DSK00006011,OFF00001002,C,M,CHF,True,E,False,True
freq,1,10,6458,5260,5307,5307,3667,5734,2437,6282


In [40]:
# Look at the description of the test data object features
test.describe(include = [np.object])

,portfolio_id,desk_id,office_id,pf_category,country_code,currency,indicator_code,type,hedge_value,status
count,4801,2853,4801,4801,4801,4801,1950,4801,1950,3344
unique,4801,2197,2,4,5,5,1,7,2,1
top,PF00003031,DSK00005682,OFF00001002,C,M,CHF,True,E,False,True
freq,1,7,3300,2675,2696,2696,1950,2940,1280,3344


### Descussion


1. **portfolio_id** - each value of this feature is different from the rest. This means that it will give no useful information when fitting a model, thus we will ignore it at that stage.  
2. **desk_id** is in the form "DSK" followed by 8 digits. Fill in the missing values with *"DSK00000000"* then delete the first 3 letters and convert the column into the float.    
3. **indicator_code** and **status** have only *True* values encoded. We will fill the rest of the values with *False*.  
4. **hedge_value** has *True* and *False* values, we will fill in the missing values with *"other"*. We will remember return to this column and create dummy variables from it as the later RandomForest algorithm will have a hard time with "other".
5. **libor_rate** is not an object that is why it is not included in the description above. We will handle its missing values by filling in the median of the rest of the values in **libor_rate**.  
6. **sold** and **bought** both have two missing values. If these values are in the same rows, we will delete those rows.  
7. And finally, we will do one-hot encoding (create dummy variables for each of the object columns).  
8. Two features, **pf_category** and **type** have a different number of unique values in training set compared to test set.  

# Data Preprocessing

## Date Columns

In [41]:
train['start_date'] = pd.to_datetime(train['start_date'], format = "%Y%m%d")
train['creation_date'] = pd.to_datetime(train['creation_date'], format = '%Y%m%d')
train['sell_date'] = pd.to_datetime(train['sell_date'], format = '%Y%m%d')

test['start_date'] = pd.to_datetime(test['start_date'], format = "%Y%m%d")
test['creation_date'] = pd.to_datetime(test['creation_date'], format = "%Y%m%d")
test['sell_date'] = pd.to_datetime(test['sell_date'], format = "%Y%m%d")

# New Features from Date Columns

In [42]:
train['start_creation'] = (train.creation_date - train.start_date).astype('timedelta64[h]')
train['creation_sell'] = (train.sell_date - train.creation_date).astype('timedelta64[h]')

test['start_creation'] = (test.creation_date - test.start_date).astype('timedelta64[h]')
test['creation_sell'] = (test.sell_date - test.creation_date).astype('timedelta64[h]')

# New Feature from *sold* and *bought* Columns 

**Note**: With this feature added, the leaderboard score is dropping from 0.96169 to 0.95306 (randomforest model)

In [43]:
#train['profit'] = train.bought - train.sold
#test['profit'] = test.bought - test.sold

## Filling in Missing Values

In [44]:
train_new = train.drop(['portfolio_id', 'start_date', 'creation_date', 'sell_date'], axis = 1)

train_new['desk_id'].fillna('DSK00000000', inplace = True)
train_new['desk_id'] = train_new['desk_id'].str[3:].astype(float)

train_new['libor_rate'].fillna(train_new['libor_rate'].median(), inplace = True)
train_new['indicator_code'].fillna(False, inplace = True)
train_new['hedge_value'].fillna('other', inplace = True)
train_new['status'].fillna(False, inplace = True)

# Check if the two missing values of sold and two missing values of bought are in the same rows
len(train_new) == sum(train_new.isnull().sold == train_new.isnull().bought)

True

In [45]:
# As 2 missing values of sold and bought are in the same rows, remove those tows
train_new.dropna(axis = 0, how = 'any', inplace = True)
print(train_new.shape)
train_new.dtypes

(9364, 16)


desk_id           float64
office_id          object
pf_category        object
sold              float64
country_code       object
euribor_rate      float64
currency           object
libor_rate        float64
bought            float64
indicator_code       bool
type               object
hedge_value        object
status               bool
return            float64
start_creation    float64
creation_sell     float64
dtype: object

In [46]:
# Do the same transformations for test data

# First keep the id column in an array so that we can refer it later
sub_ids = test['portfolio_id']

test_new = test.drop(['portfolio_id', 'start_date', 'creation_date', 'sell_date'], axis = 1)

test_new['desk_id'].fillna('DSK00000000', inplace = True)
test_new['desk_id'] = test_new['desk_id'].str[3:].astype(float)

test_new['libor_rate'].fillna(train_new['libor_rate'].median(), inplace = True)
test_new['indicator_code'].fillna(False, inplace = True)
test_new['hedge_value'].fillna('other', inplace = True)
test_new['status'].fillna(False, inplace = True)

## Dealing with Different Number of Categories (*pf_category* and *type* features)

In [47]:
print(train_new['pf_category'].unique())
print(test_new['pf_category'].unique())

['B' 'A' 'C' 'D' 'E']
['A' 'B' 'C' 'D']


In [48]:
print(train_new['type'].unique())
print(test_new['type'].unique())

['B' 'C' 'A' 'D' 'E' 'F' 'G' 'H']
['A' 'B' 'C' 'D' 'E' 'F' 'H']


In [49]:
# Delete rows of training set that correspond to the value "E" of pf_category 
train_new = train_new[~train_new['pf_category'].str.contains("E")]

# Delete rows of training set that correspond to the value "G" of type
train_new = train_new[~train_new['type'].str.contains("G")]

# Look at the descriptions of training and test sets again
train_new.describe(include = [np.object])

,office_id,pf_category,country_code,currency,type,hedge_value
count,9361,9361,9361,9361,9361,9361
unique,2,4,5,5,7,3
top,OFF00001002,C,M,CHF,E,other
freq,6458,5260,5307,5307,5732,5696


In [50]:
test_new.describe(include = [np.object])

,office_id,pf_category,country_code,currency,type,hedge_value
count,4801,4801,4801,4801,4801,4801
unique,2,4,5,5,7,3
top,OFF00001002,C,M,CHF,E,other
freq,3300,2675,2696,2696,2940,2851


## One-hot Encoding

In [51]:
print('train_new shape:', train_new.shape, 'test_new shape:', test_new.shape)

train_dummies = pd.get_dummies(train_new, prefix = ['office_id', 'pf_category', 'country_code', 'currency', 'type', 'hedge_value'])
print('train_dummies Shape', train_dummies.shape)
print(train_dummies.dtypes)
print('---------------------------------')

test_dummies = pd.get_dummies(test_new, prefix = ['office_id', 'pf_category', 'country_code', 'currency', 'type', 'hedge_value'])
print('test_dummies Shape', test_dummies.shape)
print(test_dummies.dtypes)

train_new shape: (9361, 16) test_new shape: (4801, 15)
train_dummies Shape (9361, 36)
desk_id                  float64
sold                     float64
euribor_rate             float64
libor_rate               float64
bought                   float64
indicator_code              bool
status                      bool
return                   float64
start_creation           float64
creation_sell            float64
office_id_OFF00001001      uint8
office_id_OFF00001002      uint8
pf_category_A              uint8
pf_category_B              uint8
pf_category_C              uint8
pf_category_D              uint8
country_code_M             uint8
country_code_N             uint8
country_code_T             uint8
country_code_U             uint8
country_code_Z             uint8
currency_CHF               uint8
currency_EUR               uint8
currency_GBP               uint8
currency_JPY               uint8
currency_USD               uint8
type_A                     uint8
type_B                 

# Visualizing Data

In [52]:
# Do this later

import matplotlib.pyplot as plt
%matplotlib notebook

#plt.figure()
#plt.plot(train.start_date, train.euribor_rate, '--r')
#plt.plot(train.start_date, train.libor_rate, 'o')
#plt.plot(train.start_date, train.sold, '.')


# Random Forest

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

y_train = train_dummies['return']
X_train = train_dummies.drop('return', axis = 1)

# split data into training and validation sets (for later use)
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state = 3)

forest_reg = RandomForestRegressor(random_state=0)
scores = cross_val_score(forest_reg, X_train, y_train, scoring = 'r2', cv=5)
print(scores)
print('mean r2:',np.mean(scores))

[-0.37544324  0.76566659  0.93452145  0.38797311  0.84544306]
mean r2: 0.511632194695


#  Predict on test Data (labeled)

In [54]:
forest_reg = RandomForestRegressor(random_state = 0).fit(X_train, y_train)
#forest_reg.score(X_test, y_test)

# Predict on test Data (unseen)

In [55]:
from IPython.display import FileLink


predictions = forest_reg.predict(test_dummies)

sub = pd.DataFrame({'portfolio_id': sub_ids, 'return': predictions})
filename = 'sub_returns.csv'
sub.to_csv(filename, index=False)
FileLink(filename) # leaderboard 0.96169

/Users/araks/Documents/AS_projects/Competitions/hackerearth/BrainWaves2017_18/Predict_Annual_Returns/sub_returns.csv